In [2]:
from selenium.webdriver import Chrome
import argparse
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from src.homes import *
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

In [3]:
def get_addresses(city, state, max_price):
    # Set up Selenium Chrome driver
    client = Redfin()
    city_id = client.search(city)['payload']['sections'][0]['rows'][0]['id'].split("_")[1]
    options = Options()
    options.add_argument("--headless")  # Run Chrome in headless mode (no GUI)
    driver = webdriver.Chrome(options=options)

    # Navigate to Redfin search page
    if city is None or state is None or max_price is None:
        return "Please enter a city, state, and max price"
    city = city.replace(" ", "-")
    search_url = f"https://www.redfin.com/city/{city_id}/{state}/{city}/filter/max-price={max_price}"
    print(search_url)
    driver.get(search_url)
    not_at_end = True
    addresses = []
    # Wait for the page to load and retrieve all house HTML elements
    while not_at_end:
        driver.implicitly_wait(5)  # Increase the wait time if needed
        house_elements = driver.find_elements(By.CSS_SELECTOR, ".MapHomeCardReact.HomeCard")
        print(house_elements)
        # Extract addresses from each house HTML element
        for element in house_elements:
            html = element.get_attribute("innerHTML")
            soup = BeautifulSoup(html, 'html.parser')
            address_span = soup.find('div', {'class': 'link-and-anchor'})
            if address_span is not None:
                address = address_span.text.strip()
                addresses.append(address)
            

        html = driver.page_source
        
        # Parse the HTML with BeautifulSoup
        soup = BeautifulSoup(html, 'html.parser')
        
        # Find the current page element
        current_page = soup.find('a', {'class': 'selected'})
        
        # Get the URL of the next page
        next_page = current_page.find_next_sibling('a')
        
        if next_page is None:
            print("Last page reached")
            not_at_end = False
            break
        
        # Get the URL of the next page
        next_page_url = next_page['href']
        next_page_url = f"https://www.redfin.com{next_page_url}"
        next_page_url = next_page_url.replace("/page-", f"/filter/max-price={max_price}/page-")
        
        # Navigate to the next page
        print(next_page_url)
        driver.get(next_page_url)
        print("Navigating to Next Page")
        
        
        
        # Close the browser
    driver.quit()

    return addresses


# Example usage
city = "San Francisco"
state = "CA"
max_price = 600000
addresses = get_addresses(city, state, max_price)
print(addresses)


https://www.redfin.com/city/17151/CA/San-Francisco/filter/max-price=600000
[<selenium.webdriver.remote.webelement.WebElement (session="fe84c9a9a12bd97ca9422a40624e1d6b", element="5E9A510856B65035E689CE74E2809301_element_57")>, <selenium.webdriver.remote.webelement.WebElement (session="fe84c9a9a12bd97ca9422a40624e1d6b", element="5E9A510856B65035E689CE74E2809301_element_58")>, <selenium.webdriver.remote.webelement.WebElement (session="fe84c9a9a12bd97ca9422a40624e1d6b", element="5E9A510856B65035E689CE74E2809301_element_59")>, <selenium.webdriver.remote.webelement.WebElement (session="fe84c9a9a12bd97ca9422a40624e1d6b", element="5E9A510856B65035E689CE74E2809301_element_60")>, <selenium.webdriver.remote.webelement.WebElement (session="fe84c9a9a12bd97ca9422a40624e1d6b", element="5E9A510856B65035E689CE74E2809301_element_61")>, <selenium.webdriver.remote.webelement.WebElement (session="fe84c9a9a12bd97ca9422a40624e1d6b", element="5E9A510856B65035E689CE74E2809301_element_62")>, <selenium.webdrive

In [4]:
homes = Homes()

In [5]:

for address in addresses:
    print(address)
    homes.add_info_from_address(address)
    

homes.get_data()

650 Turk St #306, San Francisco, CA 94102
750 Van Ness Ave #402, San Francisco, CA 94102
88 Arkansas St #305, San Francisco, CA 94107
88 Arkansas St #219, San Francisco, CA 94107
960 Bush St #312, San Francisco, CA 94109
1 Daniel Burnham Ct #920, San Francisco, CA 94109
631 Ofarrell St #807, San Francisco, CA 94109
1201 Sutter St #209, San Francisco, CA 94109
901 Bush St #405, San Francisco, CA 94109
601 Van Ness Ave #242, San Francisco, CA 94102
4431 - B 20th St, San Francisco, CA 94114
2162 Pine St #203, San Francisco, CA 94115
520 Natoma St #6, San Francisco, CA 94103
555 Innes Ave #309, San Francisco, CA 94124
901 Bush St #406, San Francisco, CA 94109
2415 Van Ness Ave #205, San Francisco, CA 94109
1000 North Pt #207, San Francisco, CA 94109
1264 Bush St #5, San Francisco, CA 94109
970 Duncan St Unit F-107, San Francisco, CA 94131
501 Crescent Way #5303, San Francisco, CA 94134
240 Lombard St #235, San Francisco, CA 94111
881 Moultrie St, San Francisco, CA 94110
6 Mint Plz Unit 701

,address,city,state,zip_code,beds,baths,zestimate,last_price,parking,parking_type,sq_ft,lot_size,home_type
0,650 Turk St,San Francisco,CA,94102,0,1.0,"$398,975",N/A,None,None,373,None,New
1,750 Van Ness Ave,San Francisco,CA,94102,0,1.0,"$399,579 (-$80K since last sold)",480000,None,None,391,None,New
2,88 Arkansas St,San Francisco,CA,94107,1,1.0,Estimate Not Available,N/A,None,None,502,None,New
3,88 Arkansas St,San Francisco,CA,94107,0,1.0,Estimate Not Available,N/A,None,None,374,None,New
4,960 Bush St,San Francisco,CA,94109,0,1.0,Estimate Not Available,N/A,1,"37,816",560,None,New
...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,890 Moultrie St,San Francisco,CA,94110,None,None,"$257,866",N/A,None,None,None,None,New
99,886 Moultrie St,San Francisco,CA,94110,None,None,"$212,104",N/A,None,None,None,None,New
100,99 Rausch St,San Francisco,CA,94103,1,1.0,Estimate Not Available,N/A,None,None,609,None,New
101,60 Ora Way,San Francisco,CA,94131,0,1.0,"$293,463 (+$55K since last sold)",237536,1,"6,813",409,None,New


In [9]:
homes.query_r('cash_on_cash_return', '>0')

,address,purchase_price,down%,down_payment,closing_costs,loan_amount,loan_term,interest_rate,monthly_mortgage,monthly_hoa,property_tax_rate,monthly_property_tax,monthly_home_insurance,monthly_rent,annural_rent,monthly_expenses,monthly_cash_flow,annual_cash_flow,cap_rate,cash_on_cash_return
28,1739 19th Ave,486536.60,20.0,97307.32,6000,395229.28,30,7.378,2730.558550,330.0,1.25,506.808958,129.743093,4780.0,57360.0,3697.110601,1082.889399,12994.672784,9.290959,12.578656
94,4800 3rd St,267095.65,20.0,53419.13,6000,219676.52,30,7.378,1517.700308,507.0,1.25,278.224635,71.225507,2446.0,29352.0,2374.150450,71.849550,862.194603,6.984585,1.451039
